# Generating Word Embeddings - Lab

## Introduction

In this lab, you'll learn how to generate word embeddings by training a Word2Vec model, and then embedding layers into deep neural networks for NLP!

## Objectives

You will be able to:

- Train a Word2Vec model and transform words into vectors 
- Obtain most similar words by using methods associated with word vectors 


## Getting Started

In this lab, you'll start by creating your own word embeddings by making use of the Word2Vec model. Then, you'll move onto building neural networks that make use of **_Embedding Layers_** to accomplish the same end-goal, but directly in your model. 

As you've seen, the easiest way to make use of Word2Vec is to import it from the [Gensim Library](https://radimrehurek.com/gensim/). This model contains a full implementation of Word2Vec, which you can use to begin training immediately. For this lab, you'll be working with the [News Category Dataset from Kaggle](https://www.kaggle.com/rmisra/news-category-dataset/version/2#_=_).  This dataset contains headlines and article descriptions from the news, as well as categories for which type of article they belong to.

Run the cell below to import everything you'll need for this lab. 

In [1]:
import pandas as pd
import numpy as np
np.random.seed(0)
from gensim.models import Word2Vec
from nltk import word_tokenize

C:\Users\zazue\AppData\Local\Continuum\anaconda3\envs\learn-env\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Now, import the data. The data is stored in the file `'News_Category_Dataset_v2.json'`.  This file is compressed, so that it can be more easily stored in a GitHub repo. **_Make sure to unzip the file before continuing!_**

In the cell below, use the `read_json()` function from Pandas to read the dataset into a DataFrame. Be sure to include the parameter `lines=True` when reading in the dataset!

Once you've imported the data, inspect the first few rows of the DataFrame to see what your data looks like. 

In [3]:
df = pd.read_json('News_Category_Dataset_v2.json', lines=True)
df.head()

,authors,category,date,headline,link,short_description
0,Melissa Jeltsen,CRIME,2018-05-26,There Were 2 Mass Shootings In Texas Last Week...,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...
1,Andy McDonald,ENTERTAINMENT,2018-05-26,Will Smith Joins Diplo And Nicky Jam For The 2...,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.
2,Ron Dicker,ENTERTAINMENT,2018-05-26,Hugh Grant Marries For The First Time At Age 57,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...
3,Ron Dicker,ENTERTAINMENT,2018-05-26,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...
4,Ron Dicker,ENTERTAINMENT,2018-05-26,Julianna Margulies Uses Donald Trump Poop Bags...,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ..."


## Preparing the Data

Since you're working with text data, you need to do some basic preprocessing including tokenization. Notice from the data sample that two different columns contain text data -- `headline` and `short_description`. The more text data your Word2Vec model has, the better it will perform. Therefore, you'll want to combine the two columns before tokenizing each comment and training your Word2Vec model. 

In the cell below:

* Create a column called `'combined_text'` that consists of the data from the `'headline'` column plus a space character (`' '`) plus the data from the `'short_description'` column 
* Use the `'combined_text'` column's `.map()` method and pass in `word_tokenize`. Store the result returned in `data` 

In [4]:
df['combined_text'] = df['headline'] + ' ' + df['short_description']
data = df['combined_text'].map(word_tokenize)

Inspect the first 5 items in `data` to see how everything looks. 

In [7]:
data[:5]

0    [There, Were, 2, Mass, Shootings, In, Texas, L...
1    [Will, Smith, Joins, Diplo, And, Nicky, Jam, F...
2    [Hugh, Grant, Marries, For, The, First, Time, ...
3    [Jim, Carrey, Blasts, 'Castrato, ', Adam, Schi...
4    [Julianna, Margulies, Uses, Donald, Trump, Poo...
Name: combined_text, dtype: object

Notice that although the words are tokenized, they are still in the same order they were in as headlines. This is important, because the words need to be in their original order for Word2Vec to establish the meaning of them. Remember that for a Word2Vec model you can specify a  **_Window Size_** that tells the model how many words to take into consideration at one time. 

If your window size was 5, then the model would start by looking at the words "Will Smith joins Diplo and", and then slide the window by one, so that it's looking at "Smith joins Diplo and Nicky", and so on, until it had completely processed the text example at index 1 above. By doing this for every piece of text in the entire dataset, the Word2Vec model learns excellent vector representations for each word in an **_Embedding Space_**, where the relationships between vectors capture semantic meaning (recall the vector that captures gender in the previous "king - man + woman = queen" example you saw).

Now that you've prepared the data, train your model and explore a bit!

## Training the Model

Start by instantiating a Word2Vec Model from `gensim`. 

In the cell below:

* Create a `Word2Vec` model and pass in the following arguments:
    * The dataset we'll be training on, `data`
    * The size of the word vectors to create, `size=100`
    * The window size, `window=5`
    * The minimum number of times a word needs to appear in order to be counted in  the model, `min_count=1` 
    * The number of threads to use during training, `workers=4`

In [8]:
model = Word2Vec(data, size=100, window=5, min_count=1, workers=4)

Now, that you've instantiated Word2Vec model, train it on your text data. 

In the cell below:

* Call the `.train()` method on your model and pass in the following parameters:
    * The dataset we'll be training on, `data`
    * The `total_examples`  of sentences in the dataset, which you can find in `model.corpus_count` 
    * The number of `epochs` you want to train for, which we'll set to `10`

In [9]:
model.train(data, total_examples=model.corpus_count, epochs=10)

(55565225, 67352790)

Great! You now have a fully trained model! The word vectors themselves are stored in the `Word2VecKeyedVectors` instance, which is stored in the `.wv` attribute. To simplify this, restore this object inside of the variable `wv` to save yourself some keystrokes down the line. 

In [12]:
wv = model.wv

## Examining Your Word Vectors

Now that you have a trained Word2Vec model, go ahead and explore the relationships between some of the words in the corpus! 

One cool thing you can use Word2Vec for is to get the most similar words to a given word. You can do this by passing in the word to `wv.most_similar()`. 

In the cell below, try getting the most similar word to `'Texas'`.

In [13]:
wv.most_similar('Texas')

[('Pennsylvania', 0.8209378123283386),
 ('Connecticut', 0.8149698376655579),
 ('Ohio', 0.8025277256965637),
 ('Oregon', 0.8020498752593994),
 ('Louisiana', 0.796149730682373),
 ('Maryland', 0.795978844165802),
 ('Georgia', 0.7919425368309021),
 ('Illinois', 0.791580319404602),
 ('Massachusetts', 0.7847889065742493),
 ('Wisconsin', 0.7827305793762207)]

Interesting! All of the most similar words are also states. 

You can also get the least similar vectors to a given word by passing in the word to the `.most_similar()` method's `negative` parameter. 

In the cell below, get the least similar words to `'Texas'`.

In [14]:
wv.most_similar(negative='Texas')

[('Admon', 0.43244147300720215),
 ('Animal-rescue', 0.40785229206085205),
 ('Profit-Minded', 0.4058719873428345),
 ('Optimizing', 0.4004233479499817),
 ('Uglies', 0.39972585439682007),
 ('Veers', 0.37632834911346436),
 ('Parent/Grandparent', 0.36660826206207275),
 ('Post-DOMA', 0.3615798056125641),
 ('Decoy', 0.36014819145202637),
 ('Lunatics', 0.36001890897750854)]

This seems like random noise. It is a result of the way Word2Vec is computing the similarity between word vectors in the embedding space. Although the word vectors closest to a given word vector are almost certainly going to have similar meaning or connotation with your given word, the word vectors that the model considers 'least similar' are just the word vectors that are farthest away, or have the lowest cosine similarity. It's important to understand that while the closest vectors in the embedding space will almost certainly share some level of semantic meaning with a given word, there is no guarantee that this relationship will hold at large distances. 

You can also get the vector for a given word by passing in the word as if you were passing in a key to a dictionary. 

In the cell below, get the word vector for `'Texas'`.

In [15]:
wv['Texas']

array([-0.2533224 , -1.0724185 , -1.4589592 , -0.5363621 ,  1.4013526 ,
       -0.97414404, -1.7567979 ,  0.5850633 ,  0.37341854,  0.15816644,
        0.9257152 ,  1.8997226 ,  0.9711596 , -0.4194827 ,  1.5648061 ,
       -2.0666976 ,  0.07481028, -0.11742745,  0.17562866, -0.7442575 ,
        2.139896  , -0.8041175 , -2.6727445 ,  1.4975961 , -2.3291616 ,
        2.2284577 , -0.16159515, -0.66743743,  1.7830752 , -0.75725996,
        1.1236745 , -0.9902373 , -3.9270918 , -1.007789  , -1.025457  ,
       -1.670879  , -0.6442929 ,  1.0182515 ,  0.48880625, -1.582449  ,
        0.17397143,  0.29689717, -1.2946268 , -0.8726138 ,  1.3861761 ,
        0.9016319 , -1.0131483 ,  0.7790883 ,  0.46127233,  2.6156087 ,
       -0.85641843, -0.9343315 , -2.3718746 , -0.5580914 ,  0.75139266,
        1.5405241 ,  1.0496591 , -1.9722131 ,  0.76495826,  0.53705156,
       -0.82040095, -1.4139799 ,  0.99353033, -0.03452379, -1.045754  ,
        0.5931026 ,  0.42355695, -1.6647369 ,  1.2660795 ,  0.04

Now get all of the word vectors from the object at once. You can find these inside of `wv.vectors`. Try it out in the cell below.  

In [16]:
wv.vectors

array([[ 0.5856745 ,  1.0892421 ,  0.28761235, ...,  0.4426128 ,
        -0.00189683,  0.1821919 ],
       [ 0.84787303,  0.86688024,  1.1126223 , ..., -1.172862  ,
        -0.7752291 , -0.2957771 ],
       [ 1.2728875 ,  0.6119193 ,  1.0614339 , ..., -1.0886112 ,
         1.21841   ,  1.1671827 ],
       ...,
       [ 0.05887045,  0.00186621,  0.02135438, ...,  0.05724493,
        -0.01559256, -0.02096189],
       [-0.00272029,  0.02338398, -0.01077896, ...,  0.03715703,
        -0.07498311, -0.01798521],
       [-0.02153459,  0.02466124,  0.05375085, ..., -0.00236444,
         0.02363203,  0.00228543]], dtype=float32)

As a final exercise, try to recreate the _'king' - 'man' + 'woman' = 'queen'_ example previously mentioned. You can do this by using the `.most_similar()` method and translating the word analogies into an addition/subtraction formulation (as shown above). Pass the original comparison, which you are calculating a difference between, to the negative parameter, and the analogous starter you want to apply the same transformation to, to the `positive` parameter.

Do this now in the cell below. 

In [18]:
wv.most_similar(positive=['king', 'woman'], negative=['man'])

[('queen', 0.6753336191177368),
 ('reminiscent', 0.668623685836792),
 ('crown', 0.6208366751670837),
 ('princess', 0.6133483052253723),
 ('Queen', 0.5978254079818726),
 ('jewel', 0.5887179374694824),
 ('symbol', 0.5872949361801147),
 ('brunette', 0.5849142074584961),
 ('monster', 0.5763922929763794),
 ('lover', 0.5581252574920654)]

As you can see from the output above, your model isn't perfect, but 'Queen' and 'Princess' are still in the top 5. As you can see from the other word in top 5, 'reminiscent' -- your model is far from perfect. This is likely because you didn't have enough training data. That said, given the small amount of training data provided, the model still performs remarkably well! 

In the next lab, you'll reinvestigate transfer learning, loading in the weights from an open-sourced model that has already been trained for a very long time on a massive amount of data. Specifically, you'll work with the GloVe model from the Stanford NLP Group. There's not really any benefit from training the model ourselves, unless your text uses different, specialized vocabulary that isn't likely to be well represented inside an open-source model.

## Summary

In this lab, you learned how to train and use a Word2Vec model to create vectorized word embeddings!